In [7]:
#openalpr to detect liscence plate number
from openalpr import Alpr
import sys
import cv2 as cv2
import matplotlib.pyplot as plt
import time

In [8]:
config = "C:/Users/pvpan/Desktop/Kavya Docs/Summer/220/openalpr/src/build/config/openalpr.conf"
runtimedata_path = "C:/Users/pvpan/Desktop/Kavya Docs/Summer/220/openalpr/runtime_data"

In [9]:
alpr = Alpr("us", config, runtimedata_path)

In [10]:
if not alpr.is_loaded():
    print("Error loading OpenALPR")
    sys.exit(1)

In [11]:
alpr.set_top_n(20)

In [12]:
def show(image):
    plt.figure(figsize=(5, 5))
    plt.imshow(image)
    plt.show()

In [16]:
VIDEO_SOURCE = "C:/Users/pvpan/Desktop/Kavya Docs/Summer/220/carexited.mkv"
stream = cv2.VideoCapture(VIDEO_SOURCE)
count = 0
while stream.isOpened():
    success,image= stream.read()
    if not success:
        print("video done")
        break
    #show(image)
    results = alpr.recognize_ndarray(image)
    i = 0
    dictplate ={}
    plateArray = []
    Confidence = []
    for i, plate in enumerate(results['results']):
        if plate['confidence']>88 and len(plate['plate'])==7:
            topMost = plate['candidates'][0]
            current_time = time.strftime("%m-%d-%Y %H:%M:%S") 
            print(dictplate)
            if i== 0:
                dictplate[topMost['plate']] = topMost['confidence']
                print(dictplate)
            if dictplate.get(topMost['plate']) < topMost['confidence']:
                dictplate[topMost['plate']] = topMost['confidence']
            print(i)
            i = i + 1
            #print('Plate #{}: {:7s} ({:.2f}%) {}'.format(i, topMost['plate'].upper(), topMost['confidence'],current_time))
    #print(dictplate)
    if count == 20:
        break
    count = count + 1
    

    

{}
{'7RQR550': 89.15258}
0
{}
{'7RQR550': 88.973854}
0
{}
{'7RQR550': 88.93325}
0


In [74]:
results = alpr.recognize_file("/home/harshitha_yerraguntla/image.jpeg")
i = 0
for plate in results['results']:
    i += 1
    print("Plate #%d" % i)
    print("   %12s %12s" % ("Plate", "Confidence"))
    for candidate in plate['candidates']:
        prefix = "-"
        if candidate['matches_template']:
            prefix = "*"
        print("  %s %12s%12f" % (prefix, candidate['plate'], candidate['confidence']))

 # Call when completely done to release memory
alpr.unload()

Plate #1
          Plate   Confidence
  -       FDY418   90.362671
  -       EDY418   88.886734
  -        DY418   85.589310
  -       PDY418   84.336761
  -       F0Y418   81.040321
  -       FOY418   80.750854
  -       FQY418   80.307426
  -       E0Y418   79.564384
  -        FDY18   79.474152
  -       EOY418   79.274933
  -       EQY418   78.831505
  -       FBY418   78.766022
  -       FUY418   78.359894
  -        EDY18   77.998238
  -       EBY418   77.290085
  -       EUY418   76.883957
  -        0Y418   76.266960
  -        OY418   75.977501
  -       FGY418   75.782791
  -        QY418   75.534058


In [29]:
def UpdateLiscencePlateDB(platenumber,entrytime):
    #print(platenumber,time)
    current_time = time.strftime("%m-%d-%Y %H:%M:%S")
    dynamodb = boto3.resource('dynamodb',region_name='us-east-2')
    table = dynamodb.Table('LicensePlate')
    table.update_item(
    Key={
        'Licenseplate': platenumber
    },
    UpdateExpression='SET EntryTime = :val1',
    ExpressionAttributeValues={
        ':val1': entrytime
    }
    )
    table.update_item(
    Key={
       'Licenseplate': platenumber
    },
    UpdateExpression='SET ExitTime = :val1',
    ExpressionAttributeValues={
        ':val1': '07-15-2019 19:52:45'
    }
)

In [30]:
UpdateLiscencePlateDB('8SRR680',time.strftime("%m-%d-%Y %H:%M:%S") )